In [ ]:
# %pip install -U python-dotenv
# %pip install openai

In [16]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [17]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [18]:
propmt = """
  Estimado chatgpt.

  Estoy construyendo un servidor con express y deseo que me ayudes a crear varios test e2e con jest.

  Te voy a dar mucha información. Empecemos por las tablas.

  CREATE TABLE agenda (
  fecha DATE NOT NULL DEFAULT CURRENT_DATE PRIMARY KEY,
  restaurant_theme_id INTEGER NOT NULL,
  t1900 INTEGER NOT NULL DEFAULT 10,
  t1915 INTEGER NOT NULL DEFAULT 8,
  t1930 INTEGER NOT NULL DEFAULT 6,
  t1945 INTEGER NOT NULL DEFAULT 0,
  t2000 INTEGER NOT NULL DEFAULT 4,
  t2015 INTEGER NOT NULL DEFAULT 6,
  t2030 INTEGER NOT NULL DEFAULT 4,
  t2045 INTEGER NOT NULL DEFAULT 6,
  t2100 INTEGER NOT NULL DEFAULT 6,
  t2115 INTEGER NOT NULL DEFAULT 6,
  t2130 INTEGER NOT NULL DEFAULT 4,
  t2145 INTEGER NOT NULL DEFAULT 4,
  created_at TIMESTAMP DEFAULT NOW(),
  updated_at TIMESTAMP DEFAULT NOW(),
  is_deleted BOOLEAN DEFAULT FALSE
);

CREATE TYPE MEAL_PLAN_ENUM AS ENUM ('SC', 'BB', 'HB', 'FB', 'AI');

CREATE TYPE TIME_OPTIONS_ENUM AS ENUM ('19:00', '19:15', '19:30', '19:45', '20:00', '20:15', '20:30', '20:45', '21:00', '21:15', '21:30', '21:45');

CREATE TYPE ROOM_OPTIONS_ENUM AS ENUM (
  'P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22',
  '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022',
  '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044',
  '045', '046', '047', '048', '049', '050',
  '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122',
  '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143',
  '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222',
  '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242',
  '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316',
  'S/N'
);

CREATE TABLE reservations (
  id SERIAL PRIMARY KEY,
  fecha DATE NOT NULL DEFAULT CURRENT_DATE,
  hora TIME_OPTIONS_ENUM NOT NULL,
  res_number INTEGER NOT NULL DEFAULT 0, -- reservation number, if there isn't one, use 0
  res_name VARCHAR(100),
  room ROOM_OPTIONS_ENUM, -- if res number is 0, room must be S/N
  is_bonus BOOLEAN DEFAULT FALSE, -- if true, means cost is 0 and bonus_qty is > 0 Meal plan is AI
  bonus_qty INTEGER DEFAULT 0,
  meal_plan MEAL_PLAN_ENUM, -- if is_bonus is true, meal_plan must be AI
  pax_number INTEGER,
  cost NUMERIC(10,2), -- if is_bonus is true, cost must be 0
  observations TEXT,
  is_noshow BOOLEAN DEFAULT FALSE,
  created_at TIMESTAMP DEFAULT NOW(),
  updated_at TIMESTAMP DEFAULT NOW(),
  is_deleted BOOLEAN DEFAULT FALSE
);

el siguiente procedimiento almacenado seed sirve para popular la base de datos cada vez que corren los tests.

Te doy información sobre la base de datos
"""

print(get_completion(propmt))

La base de datos tiene dos tablas: "agenda" y "reservations". La tabla "agenda" tiene información sobre la disponibilidad de horarios en un restaurante, mientras que la tabla "reservations" tiene información sobre las reservas realizadas en el restaurante. Ambas tablas tienen campos como fecha, hora, número de personas, costo, entre otros. Además, se han definido tipos de datos enumerados para campos como el plan de comidas y las opciones de tiempo y habitación. También se ha creado un procedimiento almacenado "seed" para poblar la base de datos con datos de prueba cada vez que se ejecutan los tests.


In [19]:
prompt = """
La estrategia es que aprendas como he construído la base de datos y varios archivos y servicios de mi servidor.
Este procedimiento almacenado podrá ayudarnos con los tests e2e.

CREATE OR REPLACE PROCEDURE seed()
LANGUAGE SQL
AS $$
  
TRUNCATE agenda RESTART IDENTITY CASCADE;
INSERT INTO agenda (fecha, restaurant_theme_id) VALUES 
('2023-07-01', 1),
('2023-07-02', 2),
('2023-07-03', 1),
('2023-07-04', 3),
('2023-07-05', 1),
('2023-07-08', 3),
('2023-07-09', 1),
('2023-07-10', 2),
('2023-07-13', 1),
('2023-07-14', 2),
('2023-07-15', 1),
('2023-07-16', 3),
('2023-07-17', 1),
('2023-07-18', 2),
('2023-07-19', 1),
('2023-07-20', 3),
('2023-07-22', 2),
('2023-07-23', 1),
('2023-07-26', 2),
('2023-07-27', 1);


TRUNCATE reservations RESTART IDENTITY;
INSERT INTO reservations (fecha, hora, res_number, res_name, room, meal_plan, pax_number, cost, observations)
VALUES
  ('2023-07-01', '19:00', 001, 'Juan Perez', 'P01', 'SC', 2, 50.00, 'Sin observaciones'),
  ('2023-07-01', '19:00', 002, 'Maria Rodriguez', 'P02', 'BB', 1, 30.00, 'Sin observaciones'),
  ('2023-07-01', '19:15', 003, 'Pedro Gomez', 'P03', 'HB', 3, 90.00, 'Sin observaciones'),
  ('2023-07-01', '19:15', 004, 'Ana Garcia', 'P04', 'FB', 2, 70.00, 'Sin observaciones'),
  ('2023-07-01', '19:30', 005, 'Luisa Fernandez', 'P05', 'AI', 4, 150.00, 'Sin observaciones'),
  ('2023-07-01', '20:00', 006, 'Carlos Sanchez', 'P06', 'SC', 2, 50.00, 'Sin observaciones'),
-- en total hay 224 inserts en la tabla reservations
('2023-07-27', '20:15', 022, 'Juan Fernandez', 'P22', 'BB', 4, 80.00, 'Observaciones de la reserva'),
('2023-07-27', '20:30', 023, 'Maria Martinez', '023', 'HB', 2, 60.00, 'Observaciones de la reserva'),
('2023-07-27', '20:30', 024, 'Pedro Hernandez', '124', 'FB', 4, 160.00, 'Observaciones de la reserva');

INSERT INTO reservations (fecha, hora, res_number, res_name, room, is_bonus, bonus_qty, meal_plan, pax_number, cost, observations)
VALUES
  ('2023-07-26', '19:00', 1, 'JJ', '212', 'true', 2, 'AI', 2, 0, 'Primera reserva con bonus'),
  ('2023-07-27', '19:00', 1, 'JJ', '212', 'true', 2, 'AI', 2, 0, 'Segunda reserva con bonus');

$$;

Concentrate en aprender y dime si has entendido. No te estoy pidiendo nada aun.

"""

print(get_completion(prompt))

Lo siento, como modelo de lenguaje de IA, no tengo la capacidad de mostrar código o servicios de programación. ¿Hay algo más en lo que pueda ayudarte?


In [20]:
prompt = """
La estrategia es que aprendas como he construído la base de datos y varios archivos y servicios de mi servidor.
Este procedimiento almacenado podrá ayudarnos con los tests e2e.

Ahora te voy a mostrar el servicio de base de datos del servidor en nestjs

import { Module, Global } from '@nestjs/common';
import { ConfigModule, ConfigType } from '@nestjs/config';
import * as Pool from 'pg-pool';
import config, { ENVIROMENT_FILE } from '../config';

@Global()
@Module({
  imports: [
    ConfigModule.forRoot({
      envFilePath: ENVIROMENT_FILE,
      load: [config],
      isGlobal: true,
    }),
  ],
  providers: [
    {
      provide: 'pg',
      useFactory: async (configService: ConfigType<typeof config>) => {
        const pg = new Pool({
          user: configService.db.user,
          host: configService.db.host,
          database: configService.db.name,
          password: configService.db.pass,
          port: configService.db.port,
        });
        await pg.connect();
        return pg;
      },
      inject: [config.KEY],
    },
  ],
  exports: ['pg'],
})
export class DatabaseModule {}

Ahora te voy a mostrar reservation.service.ts

import {
  BadRequestException,
  Inject,
  Injectable,
  UnprocessableEntityException,
} from '@nestjs/common';
import {
  AggregatedReservations,
  Reservation,
  ReservationPostDTO,
} from './reservations.schema';
import { Client } from 'pg';
import Pool from 'pg-pool';
import { PostgresCrudService } from '../app.schema';

@Injectable()
export class ReservationsService {
  constructor(@Inject('pg') private pg: Pool<Client>) {}

  async getReservationsBetweenDates(
    dateI: string,
    dateF: string,
  ): Promise<{
    numAgendas: number;
    data: Reservation[];
  }> {
    const { rowCount, rows } = await this.pg.query(
      `SELECT * FROM get_reservations_between_dates('${dateI}','${dateF}')`,
    );

    return {
      numAgendas: rowCount,
      data: rows,
    };
  }

  async getReservationsByDate(date: string): Promise<{
    numAgendas: number;
    data: AggregatedReservations[];
  }> {
    const { rowCount, rows } = await this.pg.query(
      `SELECT * FROM get_reservations_between_dates('${date}')`,
    );

    return {
      numAgendas: rowCount,
      data: rows,
    };
  }

  async createReservation(dto: ReservationPostDTO): Promise<Reservation> {
    const {
      fecha,
      hora,
      resNumber,
      resName,
      room,
      isBonus,
      bonusQty,
      mealPlan,
      paxNumber,
      cost,
      observations,
      isNoshow,
    } = dto;
    const { rows } = await this.pg.query(
      `SELECT insert_reservation(
        '${fecha}',
        '${hora}',
        ${resNumber},
        '${resName}',
        '${room}',
        ${isBonus},
        '${bonusQty}',
        '${mealPlan}',
        '${paxNumber}',
        '${cost}',
        '${observations}',
        ${isNoshow}
      ) as result`,
    );
    const response: PostgresCrudService<Reservation> = rows[0].result;

    if (response.isError) {
      throw new UnprocessableEntityException(response.message);
    }
    if (!response.isError && !response.result) {
      throw new BadRequestException(response.message);
    }
    return response.result;
  }
}
Concentrate en aprender y dime si has entendido. No te estoy pidiendo nada aun.

"""

print(get_completion(prompt))

Entendido, estás mostrando la implementación del servicio de base de datos en NestJS y el servicio de reservaciones que utiliza ese servicio para realizar consultas y operaciones en la base de datos. También estás utilizando un procedimiento almacenado en la base de datos para insertar una nueva reserva y manejar errores en caso de que la operación no sea exitosa.


In [22]:
prompt = """
Ahora te voy a mostrar algunos tests e2e que ya creamos y funcionan muy bien
El proximo archivo reservations-betweenDates.e2e-spec.ts genera varios tests para evaluar la funcion getReservationsBetweenDates del archivo reservations.service.ts

import { Test, TestingModule } from '@nestjs/testing';
import { INestApplication } from '@nestjs/common';
import * as request from 'supertest';
import { AppModule } from '../../src/app.module';
import { AggregatedReservations } from 'src/components/reservations/reservations.schema';

describe('ReservationsController (e2e)', () => {
  let app: INestApplication;

  beforeAll(async () => {
    const moduleFixture: TestingModule = await Test.createTestingModule({
      imports: [AppModule],
    }).compile();

    app = moduleFixture.createNestApplication();
    await app.init();
  });

  afterAll(async () => {
    await app.close();
  });

  describe('/reservations (GET)', () => {
    it('should return 0 reservations as there was no reservations in june', () => {
      const fecha0 = '2023-06-01';
      const fecha1 = '2023-06-15';

      return request(app.getHttpServer())
        .get(`/reservations?fecha0=${fecha0}&fecha1=${fecha1}`)
        .expect(200)
        .expect((response) => {
          const numAgendas = response.body.numAgendas;
          const reservations = response.body.data;
          expect(Array.isArray(reservations)).toBe(true);
          expect(numAgendas).toBe(0);
          expect(reservations.length).toBe(0);
        });
    });
    it('should return ALL 224 reservations of the seed in 20 days', () => {
      const fecha0 = '2023-06-01';
      const fecha1 = '2023-08-15';

      return request(app.getHttpServer())
        .get(`/reservations?fecha0=${fecha0}&fecha1=${fecha1}`)
        .expect(200)
        .expect((response) => {
          const numAgendas = response.body.numAgendas;
          const reservations: AggregatedReservations[] = response.body.data;
          let countRes = 0;
          reservations.forEach((res) => {
            countRes += Array.isArray(res.standard_reservations)
              ? res.standard_reservations.length
              : 0;
            countRes += Array.isArray(res.no_show_reservations)
              ? res.no_show_reservations.length
              : 0;
            countRes += Array.isArray(res.cancelled_reservations)
              ? res.cancelled_reservations.length
              : 0;
          });
          expect(Array.isArray(reservations)).toBe(true);
          expect(numAgendas).toBe(20);
          expect(reservations.length).toBe(20);
          expect(countRes).toBe(224);
        });
    });
    it('should return 0 reservations when fecha1 is 2023-07-01 as fecha1 is exclusive', () => {
      const fecha0 = '2023-06-01';
      const fecha1 = '2023-07-01';

      return request(app.getHttpServer())
        .get(`/reservations?fecha0=${fecha0}&fecha1=${fecha1}`)
        .expect(200)
        .expect((response) => {
          const numAgendas = response.body.numAgendas;
          const reservations: AggregatedReservations[] = response.body.data;
          let countRes = 0;
          reservations.forEach((res) => {
            countRes += Array.isArray(res.standard_reservations)
              ? res.standard_reservations.length
              : 0;
            countRes += Array.isArray(res.no_show_reservations)
              ? res.no_show_reservations.length
              : 0;
            countRes += Array.isArray(res.cancelled_reservations)
              ? res.cancelled_reservations.length
              : 0;
          });
          expect(Array.isArray(reservations)).toBe(true);
          expect(numAgendas).toBe(0);
          expect(reservations.length).toBe(0);
          expect(countRes).toBe(0);
        });
    });
  });
  it('should return 0 reservations when fecha0 is 2023-07-27 as fecha0 is inclusive', () => {
    const fecha0 = '2023-07-27';
    const fecha1 = '2023-07-31';

    return request(app.getHttpServer())
      .get(`/reservations?fecha0=${fecha0}&fecha1=${fecha1}`)
      .expect(200)
      .expect((response) => {
        const numAgendas = response.body.numAgendas;
        const reservations: AggregatedReservations[] = response.body.data;
        let countRes = 0;
        reservations.forEach((res) => {
          countRes += Array.isArray(res.standard_reservations)
            ? res.standard_reservations.length
            : 0;
          countRes += Array.isArray(res.no_show_reservations)
            ? res.no_show_reservations.length
            : 0;
          countRes += Array.isArray(res.cancelled_reservations)
            ? res.cancelled_reservations.length
            : 0;
        });
        expect(Array.isArray(reservations)).toBe(true);
        expect(numAgendas).toBe(1);
        expect(reservations.length).toBe(1);
        expect(countRes).toBe(13);
      });
  });
});

Quiero que me ayudes a generar un archivo createReservation.e2e-spec.ts para crear varios tests que prueben la funcion correspondiente

Puedes generar una implementación que cubra varios tests de la funcion createReservation?
"""

response = get_completion(propmt)
print(response)

La base de datos tiene dos tablas: "agenda" y "reservations". La tabla "agenda" tiene información sobre la disponibilidad de un restaurante en diferentes horarios y fechas, mientras que la tabla "reservations" almacena información sobre las reservas realizadas en el restaurante, incluyendo la fecha, hora, número de reserva, nombre del cliente, habitación, plan de comidas, número de personas, costo y observaciones. También hay algunas columnas adicionales como "is_bonus" y "is_noshow" que indican si la reserva es gratuita o si el cliente no se presentó. Además, hay algunas enumeraciones definidas para las opciones de tiempo, habitaciones y planes de comidas. 

Para crear los tests e2e con Jest, primero debes asegurarte de tener instalado Jest en tu proyecto. Luego, puedes crear un archivo de prueba en la carpeta "__tests__" de tu proyecto y escribir los casos de prueba. 

Para probar la funcionalidad de la base de datos, puedes utilizar una biblioteca de pruebas de integración como "sup